In [2]:
from torch_geometric.nn.models import tgn

In [ ]:
import torch
import numpy as np

# Example distance matrix
D = np.array([
    [0.0, 0.5, 2.1],
    [0.5, 0.0, 1.7],
    [2.1, 1.7, 0.0]
])

# Threshold to determine edges
epsilon = 1.0

# Build edge list based on the threshold
src, dst, weights = [], [], []

for i in range(D.shape[0]):
    for j in range(D.shape[1]):
        if i != j and D[i][j] <= epsilon:
            src.append(i)
            dst.append(j)
            weights.append(1.0 / (1e-5 + D[i][j]))  # optional: inverse distance as weight

# so to represent the adjacency matrix you use edge list and the other the weights?
edge_index = torch.tensor([src, dst], dtype=torch.long)
edge_weight = torch.tensor(weights, dtype=torch.float32)


In [4]:
x = torch.randn((D.shape[0], 16))  # 3 nodes, 16 features each


In [5]:
from torch_geometric.nn import GCNConv

conv = GCNConv(in_channels=16, out_channels=32)

# Forward pass
x_out = conv(x, edge_index, edge_weight)


In [6]:
print(x_out)

tensor([[-3.2089e-01, -6.3288e-01, -1.8896e-01, -2.3628e-01, -2.8817e-01,
          3.4328e-01,  6.7342e-01, -2.4728e-01, -3.5239e-01, -2.4342e-01,
          3.2315e-01,  5.0067e-01, -2.5791e-01,  6.0807e-01, -4.8137e-01,
          4.2984e-02, -3.7144e-01, -4.0818e-02,  4.5687e-01,  1.8449e-02,
         -1.5940e-01, -8.6577e-02,  7.2570e-01,  9.0183e-01, -3.8834e-01,
          9.1921e-02, -2.5331e-01,  3.8133e-01, -2.4998e-01, -1.9012e-01,
          3.1951e-02, -4.5143e-01],
        [-3.8717e-01, -5.7393e-01, -1.9860e-01,  2.0425e-01, -2.2687e-01,
          3.6155e-01,  7.2146e-01, -3.6963e-01, -5.7922e-01, -1.8449e-01,
          2.6313e-01,  1.2110e-01, -4.8737e-01,  8.1585e-02, -4.0970e-01,
          1.6158e-01, -1.6580e-01,  3.4525e-01,  2.5812e-01, -9.0533e-02,
         -5.5841e-02, -2.6969e-02,  6.7868e-01,  8.4918e-01, -3.6789e-02,
          8.3061e-01, -1.2569e-01,  2.9951e-02, -5.1312e-01, -1.9402e-01,
          6.2289e-02, -4.6807e-01],
        [-1.9891e+00,  6.9298e-01, -6.33